# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [3]:
!pip install pymongo

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']


In [6]:
# review the collections in our new database
print(db.list_collection_names())

[]


In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

None


In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [46]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}
   


In [47]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('6772dd48bd801ad0ec8ec76e'), acknowledged=True)

In [40]:
# Define the restautant to check
restaurant_name = "Penang Flavours"

In [41]:
# Query the collection to check if the restaurant exists
new_restaurant = establishments.find_one({"Penang Flavours": restaurant_name})

In [48]:
# check and print the results
if new_restaurant:
    print(f"Restaurant '{restaurant_name}' exists in the database!")
    print("Details:", new_restaurant)
else:
    print(f"Restaurant '{restaurant_name}' was not found in the database.")

Restaurant 'Penang Flavours' exists in the database!
Details: {'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True, '_id': ObjectId('6772dd48bd801ad0ec8ec76e')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [51]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}
pprint(establishments.find_one(query,projection=fields))

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 '_id': ObjectId('6772d72dbd801ad0ec8ec76c')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [65]:
# Update the new restaurant with the correct BusinessTypeID
result = establishments.update_one(
    {"BusinessName":"Penang Flavours"},
    {"$set": {"BusinessTypeID": 1}}
)

                           

In [60]:
# check the result
if result.modified_count > 0:
    print("Update successful!")
else:
    print("No document matched the query or the value was already set.")

Update successful!


In [64]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName":"Penang Flavours"}))


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6772d72dbd801ad0ec8ec76c'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [82]:
# Find how many documents have LocalAuthorityName as "Dover"
count = establishments.count_documents({"LocalAuthorityName": "Dover"})

print(f"Number of documents with LocalAuthorityName as 'Dover': {count}")


Number of documents with LocalAuthorityName as 'Dover': 0


In [77]:
# Delete all documents where LocalAuthorityName is "Dover"
result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Number of documents deleted: {result.deleted_count}")

Number of documents deleted: 0


In [81]:
# Check if any remaining documents include Dover
remaining_document = establishments.find_one({"LocalAuthorityName": "Dover"})

if remaining_document:
    print("There are still documents with LocalAuthorityName as 'Dover'.")
else:
    print("No documents with LocalAuthorityName as 'Dover' remain.")


No documents with LocalAuthorityName as 'Dover' remain.


In [87]:
# Check that other documents remain with 'find_one'
remaining_document = establishments({"LocalAuthorityName": {"$ne": "Dover"}})

if remaining_document:
    print(f"There are still other documents remaining in the establishment.")
else:
    print(f"No other documents remain in the establishment.")


TypeError: 'Collection' object is not callable. If you meant to call the 'establishments' method on a 'Database' object it is failing because no such method exists.

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [92]:
# Change the data type from String to Decimal for longitude and latitude

result = collection.update_many(
    {},  # No filter to update all documents
    [
        {
            '$set': {
                "geocode.longitude": {'$toDecimal': "$geocode.longitude"},
                "geocode.latitude": {'$toDecimal': "$geocode.latitude"}
            }
        }
    ]
)

# Output the result
print(f"Number of documents updated: {result.modified_count}")


Number of documents updated: 3


Use `update_many` to convert `RatingValue` to integer numbers.

In [94]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

print(f"Number of documents updated: {result.modified_count}")

Number of documents updated: 3


In [98]:
# Change the data type from String to Integer for RatingValue

result = collection.update_many(
    {"RatingValue": {"$type": "string"}},  # Filter: Update only if RatingValue is a string
    [
        {
            '$set': {
                "RatingValue": {'$toInt': "$RatingValue"}  # Convert RatingValue to Integer
            }
        }
    ]
)



In [101]:
# Check that the coordinates and rating value are now numbers

cursor = establishments.find(
    {
        "geocode.longitude": {"$type": "double"},  # Check if longitude is a number (double/float)
        "geocode.latitude": {"$type": "double"},   # Check if latitude is a number (double/float)
        "RatingValue": {"$type": "int"}            # Check if RatingValue is an integer
    },
    {"geocode.longitude": 1, "geocode.latitude": 1, "RatingValue": 1}  # Only return relevant fields
)

# Check and print the result
documents = list(cursor)

